In [1]:
import casadi as ca
import numpy as np
import sys

sys.path.insert(0, '../python/pyecca')

import pyecca.lie.so3 as so3
from pyecca.util import rk4

def gazebo_state():
    # gazebo variables that you want to pass in
    omega_ENU = ca.SX.sym('omega_ENU', 3)
    pos_ENU = ca.SX.sym('pos_ENU', 3)
    vel_ENU = ca.SX.sym('vel_ENU', 3)
    q_ENU_DLF = ca.SX.sym('q_ENU_DLF', 4)
    m = ca.SX.sym('m')
    x_gz = ca.vertcat(omega_ENU, pos_ENU, vel_ENU, q_ENU_DLF, m)
    
    p = ca.SX.sym('p', 15)
    u = ca.SX.sym('u', 4)

    C_NED_ENU = np.array([
        #E  N   U
        [0, 1,  0], # N
        [1, 0,  0], # E
        [0, 0, -1]  # D
    ])

    C_DLF_FRD = np.array([
        #F   R  D
        [0,  0, 1],  # D
        [0, -1, 0],  # L
        [1,  0, 0]   # F
    ])
    
    C_ENU_DLF = so3.Dcm.from_quat(q_ENU_DLF)
    C_FRD_ENU = ca.mtimes(C_DLF_FRD.T, C_ENU_DLF.T)
    
    r_NED_FRD = so3.Mrp.from_dcm(
        ca.mtimes([C_NED_ENU, C_ENU_DLF, C_DLF_FRD]))
    
    omega_FRD = ca.mtimes(C_FRD_ENU, omega_ENU)
    pos_NED = ca.mtimes(C_NED_ENU, pos_ENU)
    vel_FRD = ca.mtimes(C_FRD_ENU, vel_ENU)
    
    x = ca.vertcat(omega_FRD, r_NED_FRD, vel_FRD, pos_NED, m)
    
    return ca.Function('gazebo_state', [x_gz], [x])

In [2]:
def rocket_equations(jit=True):
    x = ca.SX.sym('x', 14)
    u = ca.SX.sym('u', 4)
    p = ca.SX.sym('p', 15)

    # State: x
    omega_b = x[0:3]  # inertial angular velocity expressed in body frame
    r_nb = x[3:7]  # modified rodrigues parameters
    v_b = x[7:10]  # inertial velocity expressed in body components
    p_n = x[10:13]  # positon in nav frame
    m_fuel = x[13]  # mass
    
    # Input: u
    m_dot = ca.if_else(m_fuel > 0, u[0], 0)
    aileron = u[1]
    elevator = u[2]
    rudder = u[3]
    
    # Parameters: p
    g = p[0]  # gravity
    Jx = p[1]  # moment of inertia
    Jy = p[2]
    Jz = p[3]
    Jxz = p[4]
    ve = p[5]
    l_fin = p[6]
    CL_alpha = p[7]
    CL0 = p[8]
    CD0 = p[9]
    K = p[10]
    s_fin = p[11]
    rho = p[12]
    m_empty = p[13]
    l_motor = p[14]
    
    # Calculations
    m = m_empty + m_fuel
    J_b = ca.SX.zeros(3, 3)
    J_b[0, 0] = Jx + m_fuel*l_motor**2
    J_b[1, 1] = Jy + m_fuel*l_motor**2
    J_b[2, 2] = Jz
    J_b[0, 2] = J_b[2, 0] = Jxz
    C_nb = so3.Dcm.from_mrp(r_nb)
    g_n = ca.vertcat(0, 0, g)
    v_n = ca.mtimes(C_nb, v_b)
    
    # aerodynamics
    VT = ca.norm_2(v_b)
    q = 0.5*rho*VT**2
    fins = {
        'top': {
            'fwd': [1, 0, 0],
            'up': [0, 1, 0],
            'mix': aileron + rudder,
        },
        'left': {
            'fwd': [1, 0, 0],
            'up': [0, 0, -1],
            'mix': aileron + elevator,
        },
        'down': {
            'fwd': [1, 0, 0],
            'up': [0, -1, 0],
            'mix': aileron - rudder,
        },
        'right': {
            'fwd': [1, 0, 0],
            'up': [0, 0, 1],
            'mix': aileron - elevator,
        },
    }
    vel_tol = 1e-3
    FA_b = ca.SX.zeros(3)
    MA_b = ca.SX.zeros(3) 
    for key, data in fins.items():
        fwd = data['fwd']
        up = data['up']
        mix = data['mix']
        U = ca.dot(fwd, v_b)
        W = ca.dot(up, v_b)
        alpha = ca.if_else(
            ca.logic_and(ca.fabs(W) > vel_tol, ca.fabs(U) > vel_tol),
            -ca.atan(W/U), 0)
        rel_wind_dir = ca.if_else(ca.fabs(VT) > vel_tol, v_b/VT, -ca.DM(fwd))
        perp_wind_dir = ca.cross(ca.cross(fwd, up), rel_wind_dir)
        perp_wind_dir = perp_wind_dir/ca.norm_2(perp_wind_dir)
        CL = CL0 + CL_alpha*(alpha + mix)
        CD = CD0 + K*(CL - CL0)**2
        L = CL*q*s_fin
        D = CD*q*s_fin
        FA_b += L*perp_wind_dir - D*rel_wind_dir
        MA_b += ca.cross(ca.vertcat(-l_fin, 0, 0), FA_b)

    # propulsion
    FP_b = ca.vertcat(m_dot*ve, 0, 0)
    MP_b = ca.vertcat(0, 0, 0)
    
    # force and momental total
    F_b = FA_b + FP_b + ca.mtimes(C_nb.T, g_n)
    M_b = MA_b + MP_b
    
    rocket_aero_forces = ca.Function(
        'rocket_aero_forces', [x, u, p], [FA_b], ['x', 'u', 'p'], ['FA_b'])
    
    
    # right hand side
    rhs = ca.Function('rhs', [x, u, p], [ca.vertcat(
        ca.mtimes(ca.inv(J_b), M_b - ca.cross(omega_b, ca.mtimes(J_b, omega_b))),
        so3.Mrp.kinematics(r_nb, omega_b),
        F_b/m - m_dot*v_b/m - ca.cross(omega_b, v_b),
        ca.mtimes(C_nb, v_b), -m_dot)], ['x', 'u', 'p'], ['rhs'], {'jit': jit})

    # prediction
    t0 = ca.SX.sym('t0')
    h = ca.SX.sym('h')
    x0 = ca.SX.sym('x', 14)
    x1 = rk4(lambda t, x: rhs(x, u, p), t0, x0, h)
    x1[3:7] = so3.Mrp.shadow_if_necessary(x1[3:7])
    predict = ca.Function('predict', [x0, u, p, t0, h], [x1], {'jit': jit})

    # initialize
    pitch_deg = ca.SX.sym('pitch_deg')
    omega0_b = ca.vertcat(0, 0, 0)
    r0_nb = so3.Mrp.from_euler(ca.vertcat(0, pitch_deg*ca.pi/180, 0))
    v0_b = ca.vertcat(0, 0, 0)
    p0_n = ca.vertcat(0, 0, 0)
    m0_fuel = 0.8
    # x: omega_b, r_nb, v_b, p_n, m_fuel
    x0 = ca.vertcat(omega0_b, r0_nb, v0_b, p0_n, m0_fuel)
    #     g, Jx, Jy, Jz, Jxz, ve, l_fin, CL_alpha, CL0, CD0, K, s, rho, m_emptpy, l_motor
    p0 = [9.8, 1, 1, 1, 0.1, 350, 1.0, 2*np.pi, 0, 0.01, 0.01, 0.05, 1.225, 0.2, 1.0]
    initialize = ca.Function('initialize', [pitch_deg], [x0, p0])

    return {
        'rhs': rhs,
        'predict': predict,
        'initialize': initialize,
        'rocket_aero_forces': rocket_aero_forces,
        'x': x,
        'u': u,
        'p': p,
    }

In [3]:
eqs = rocket_equations()

In [4]:
x = ca.SX.sym('x', 14)
u = ca.SX.sym('u', 4)
p = ca.SX.sym('p', 15)
x_gz = gazebo_state()(x)
eqs['rocket_aero_forces'](x_gz, u, p)

SX(@1=0.001, @2=sq(x_9), @3=sq(x_10), @4=sq(x_11), @5=sq(x_12), @6=2, @7=(x_10*x_11), @8=(x_9*x_12), @9=(x_10*x_12), @10=(x_9*x_11), @11=((((((@2+@3)-@4)-@5)*x_6)+((@6*(@7+@8))*x_7))+((@6*(@9-@10))*x_8)), @12=(x_11*x_12), @13=(x_9*x_10), @14=((((@6*(@9+@10))*x_6)+((@6*(@12-@13))*x_7))+((((@2+@5)-@3)-@4)*x_8)), @15=(p_7*((((@1<fabs((-@11)))&&(@1<fabs(@14)))?(-atan((-(@11/@14)))):0)+(u_1+u_2))), @16=((((@6*(@7-@8))*x_6)+((((@2+@4)-@3)-@5)*x_7))+((@6*(@12+@13))*x_8)), @17=((sq(@14)+sq(@16))+sq(@11)), @18=((0.5*p_12)*@17), @19=(((p_8+@15)*@18)*p_11), @20=sqrt(@17), @21=(@1<fabs(@20)), @22=(@11/@20), @23=(@21?@22:0), @24=-1, @25=((@21?(@14/@20):0)+((!@21)?@24:0)), @26=sqrt((sq(@23)+sq(@25))), @27=(((p_9+(p_10*sq(@15)))*@18)*p_11), @28=(p_7*((((@1<fabs((-@16)))&&(@1<fabs(@14)))?(-atan((-(@16/@14)))):0)+(u_1+u_3))), @29=(((p_8+@28)*@18)*p_11), @30=(@1<fabs(@20)), @31=(@16/@20), @32=(@30?(-@31):0), @33=((@30?(@14/@20):0)+((!@30)?@24:0)), @34=sqrt((sq(@32)+sq(@33))), @35=(((p_9+(p_10*sq(@28)))*